<a href="https://colab.research.google.com/github/joris-vaneyghen/mss-jazz-playalong/blob/main/segmentation/cluster_segments.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# download our audio example
!git clone https://github.com/joris-vaneyghen/mss-jazz-playalong.git

Cloning into 'mss-jazz-playalong'...
remote: Enumerating objects: 135, done.
remote: Counting objects: 100% (135/135), done.
remote: Compressing objects: 100% (128/128), done.
remote: Total 135 (delta 67), reused 0 (delta 0), pack-reused 0 (from 0)
Receiving objects: 100% (135/135), 40.47 MiB | 9.78 MiB/s, done.
Resolving deltas: 100% (67/67), done.


In [2]:
import numpy as np
import json
import os

In [6]:
input_path = 'mss-jazz-playalong/examples'
output_path = 'mss-jazz-playalong/out/segment_and_tag'
resolution = 0.32 # resolution of EfficientAT model

In [4]:
def load_json(dir, mp3_file):
    # Replace .mp3 extension with .json
    json_file_name = mp3_file.replace('.mp3', '.json')
    file_path = os.path.join(dir, json_file_name)

    # Check if the .json file exists
    if not os.path.exists(file_path):
        return {}  # Return an empty dictionary if the .json file doesn't exist

    # Load the JSON file if it exists
    with open(file_path, 'r') as file:
        return json.load(file)

# def save_json(dir, mp3_file, data):
#     # Replace .mp3 extension with .json
#     json_file_name = mp3_file.replace('.mp3', '.json')
#     file_path = os.path.join(dir, json_file_name)

#     # Check if directory exists, create it if not
#     if not os.path.exists(dir):
#         os.makedirs(dir)

#     # Save the data to the .json file
#     with open(file_path, 'w') as file:
#         json.dump(data, file, indent=4)

def iterate_files(dir):
    for file_name in os.listdir(dir):
        if file_name.endswith('.mp3'):
            yield file_name

In [8]:
segment_lengths =  []
for mp3_file in iterate_files(input_path):
  data = load_json(output_path, mp3_file)
  if ('demucs' in data.keys() and 'segments' in data.keys()):
    segments = data['segments']
    for segment in segments:
      length = segment['end_idx'] - segment['start_idx']
      segment_lengths.append(length)
len(segment_lengths)



KeyError: 'end_idx'